In [1]:
import torch
from custom_loss import RocAucLoss
import numpy as np
import tensorflow as tf

In [2]:
y1 = [1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1]
y2 = np.random.rand(len(y1))
y_true_n = np.array(y1)
y_pred_n = np.array(y2)

y_true = torch.tensor(y1)
y_pred = torch.tensor(y2)

l = RocAucLoss()
t_res = float(l.forward(y_pred, y_true))
print("pytorch, ", t_res, "\n")

y_true_tf = tf.convert_to_tensor(y_true_n)
y_pred_tf = tf.convert_to_tensor(y_pred_n)

with tf.Session() as sess:
    pos = tf.boolean_mask(y_pred, tf.cast(y_true, tf.bool))
    neg = tf.boolean_mask(y_pred, ~tf.cast(y_true, tf.bool))
    pos = tf.expand_dims(pos, 0)
    neg = tf.expand_dims(neg, 1)
    gamma = 0.2
    p     = 3
    difference = tf.zeros_like(pos * neg) + pos - neg - gamma
    masked = tf.boolean_mask(difference, difference < 0.0)
    tf_res = sess.run(tf.reduce_sum(tf.pow(-masked, p)))
    
    assert np.allclose(tf_res, t_res)
    
    print("tensorflow, ", tf_res)

pytorch,  20.78910638054612 

tensorflow,  20.789106380546112
